<a href="https://colab.research.google.com/github/ferrry-ghsb/bigdata_6team/blob/main/%5Bksy%5DLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import os
import pandas as pd

In [41]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
#전처리 완료된 파일 불러오기
#x_train = pd.read_csv('/content/drive/MyDrive/store-sales-time-series-forecasting/preprocessed_data/x_train.csv')
#y_train = pd.read_csv('/content/drive/MyDrive/store-sales-time-series-forecasting/preprocessed_data/y_train.csv')
x_valid = pd.read_csv('/content/drive/MyDrive/store-sales-time-series-forecasting/preprocessed_data/x_valid.csv')
y_valid = pd.read_csv('/content/drive/MyDrive/store-sales-time-series-forecasting/preprocessed_data/y_valid.csv')
#train_trans = pd.read_csv('/content/drive/MyDrive/store-sales-time-series-forecasting/preprocessed_data/train_trans.csv')

In [43]:
x_valid

,date,onpromotion,dcoilwtico,family_AUTOMOTIVE,family_BABY CARE,family_BEAUTY,family_BEVERAGES,family_BOOKS,family_BREAD/BAKERY,family_CELEBRATION,family_CLEANING,family_DAIRY,family_DELI,family_EGGS,family_FROZEN FOODS,family_GROCERY I,family_GROCERY II,family_HARDWARE,family_HOME AND KITCHEN I,family_HOME AND KITCHEN II,family_HOME APPLIANCES,family_HOME CARE,family_LADIESWEAR,family_LAWN AND GARDEN,family_LINGERIE,"family_LIQUOR,WINE,BEER",family_MAGAZINES,family_MEATS,family_PERSONAL CARE,family_PET SUPPLIES,family_PLAYERS AND ELECTRONICS,family_POULTRY,family_PREPARED FOODS,family_PRODUCE,family_SCHOOL AND OFFICE SUPPLIES,family_SEAFOOD,city_Ambato,city_Babahoyo,city_Cayambe,city_Cuenca,...,store_nbr_15,store_nbr_16,store_nbr_17,store_nbr_18,store_nbr_19,store_nbr_20,store_nbr_21,store_nbr_22,store_nbr_23,store_nbr_24,store_nbr_25,store_nbr_26,store_nbr_27,store_nbr_28,store_nbr_29,store_nbr_30,store_nbr_31,store_nbr_32,store_nbr_33,store_nbr_34,store_nbr_35,store_nbr_36,store_nbr_37,store_nbr_38,store_nbr_39,store_nbr_40,store_nbr_41,store_nbr_42,store_nbr_43,store_nbr_44,store_nbr_45,store_nbr_46,store_nbr_47,store_nbr_48,store_nbr_49,store_nbr_50,store_nbr_51,store_nbr_52,store_nbr_53,store_nbr_54
0,2016-09-12,0,46.28,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2016-09-12,0,46.28,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2016-09-12,0,46.28,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2016-09-12,1,46.28,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2016-09-12,1,46.28,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610865,2017-08-15,0,47.57,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
610866,2017-08-15,1,47.57,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
610867,2017-08-15,148,47.57,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
610868,2017-08-15,8,47.57,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [44]:
x_valid = x_valid[["date", "onpromotion", "dcoilwtico"]]

In [45]:
x_valid

,date,onpromotion,dcoilwtico
0,2016-09-12,0,46.28
1,2016-09-12,0,46.28
2,2016-09-12,0,46.28
3,2016-09-12,1,46.28
4,2016-09-12,1,46.28
...,...,...,...
610865,2017-08-15,0,47.57
610866,2017-08-15,1,47.57
610867,2017-08-15,148,47.57
610868,2017-08-15,8,47.57


In [46]:
x_train = x_valid[:5000]

In [47]:
x_train

,date,onpromotion,dcoilwtico
0,2016-09-12,0,46.28
1,2016-09-12,0,46.28
2,2016-09-12,0,46.28
3,2016-09-12,1,46.28
4,2016-09-12,1,46.28
...,...,...,...
4995,2016-09-15,6,43.85
4996,2016-09-15,0,43.85
4997,2016-09-15,32,43.85
4998,2016-09-15,3,43.85


In [48]:
y_valid

,date,std_sales
0,2016-09-12,-0.324235
1,2016-09-12,-0.320623
2,2016-09-12,-0.246849
3,2016-09-12,-0.268242
4,2016-09-12,0.057357
...,...,...
610865,2017-08-15,0.071447
610866,2017-08-15,-0.184657
610867,2017-08-15,1.861044
610868,2017-08-15,-0.214959


In [49]:
y_train = y_valid[:5000]

In [50]:
x_train = x_train.set_index('date', drop=True)
y_train = y_train.set_index('date', drop=True)

In [51]:
x_train = x_train.join(y_train)

In [52]:
 x_train = x_train[:200000]

In [53]:
# import tensorflow as tf
# from tensorflow.keras.layers import LSTM, Flatten, TimeDistributed, Conv1D, MaxPooling1D
# from tensorflow.keras import Model, Sequential
# import seaborn as sns
# from __future__ import absolute_import, division, print_function, unicode_literals
# import io

In [54]:
def multivariate_data(dataset, target, start_index, end_index, history_size, target_size, step, single_step=False):
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i - history_size, i, step)
        data.append(dataset[indices])

        if single_step:
            labels.append(target[i + target_size])
        else:
            labels.append(target[i:i + target_size])
    return np.array(data), np.array(labels)

In [55]:
x_train.iloc[:, 2]

date
2016-09-12   -0.324235
2016-09-12   -0.320623
2016-09-12   -0.246849
2016-09-12   -0.268242
2016-09-12    0.057357
                ...   
2016-09-12   -0.159869
2016-09-12   -0.324235
2016-09-12   -0.323332
2016-09-12   -0.319719
2016-09-12   -0.307076
Name: std_sales, Length: 200000, dtype: float64

In [56]:
x_train.columns

Index(['onpromotion', 'dcoilwtico', 'std_sales'], dtype='object')

In [57]:
x_test = x_valid[10001:15001]
y_test = y_valid[10001:15001]
x_test = x_test.set_index('date', drop=True)
y_test = y_test.set_index('date', drop=True)
x_test = x_test.join(y_test)

In [58]:
dataset = x_train.values

In [59]:
dataset

array([[ 0.        , 46.28      , -0.32423504],
       [ 0.        , 46.28      , -0.32062261],
       [ 0.        , 46.28      , -0.24684853],
       ...,
       [ 0.        , 46.28      , -0.32333194],
       [ 0.        , 46.28      , -0.3197195 ],
       [ 0.        , 46.28      , -0.30707597]])

In [60]:
dataset[:, 2]

array([-0.32423504, -0.32062261, -0.24684853, ..., -0.32333194,
       -0.3197195 , -0.30707597])

In [61]:
dataset2 = x_test.values

In [62]:
import numpy as np

In [63]:
dataset.shape

(200000, 3)

In [64]:
TRAIN_SPLIT = len(dataset) - int(0.2 * len(dataset))
TRAIN_SPLIT

160000

In [65]:

past_history = 720
future_target = 72
STEP = 28

x_train_single, y_train_single = multivariate_data(dataset, dataset[:, 2], 0, TRAIN_SPLIT, past_history,
                                                   future_target, STEP, single_step=True)
x_val_single, y_val_single = multivariate_data(dataset, dataset[:, 2], TRAIN_SPLIT, None, past_history,
                                               future_target, STEP, single_step=True)


print('Single window of past history : {}'.format(x_train_single[0].shape))

Single window of past history : (26, 3)


In [23]:
x_train_single = x_train.iloc[:, :2]

In [27]:
y_train_single = x_train.iloc[:, 2]

In [29]:
x_train_single = x_train_single.values

In [30]:
y_train_single = y_train_single.values

In [69]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Flatten, TimeDistributed, Conv1D, MaxPooling1D
from tensorflow.keras import Model, Sequential
import seaborn as sns
from __future__ import absolute_import, division, print_function, unicode_literals
import io
BUFFER_SIZE = 10000
train_data_single = tf.data.Dataset.from_tensor_slices((x_train_single, y_train_single))
train_data_single = train_data_single.cache().shuffle(BUFFER_SIZE).batch(500).repeat()
val_data_single = tf.data.Dataset.from_tensor_slices((x_val_single, y_val_single))
val_data_single = val_data_single.shuffle(BUFFER_SIZE).batch(500).repeat()

In [70]:
single_step_model = tf.keras.models.Sequential()
single_step_model.add(tf.keras.layers.LSTM(8, input_shape=x_train_single.shape[-2:]))
single_step_model.add(tf.keras.layers.Dense(1))
single_step_model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='mae')

In [78]:
x_train_single.shape[-2:]

(26, 3)

In [79]:
EPOCHS = 10
EVALUATION_INTERVAL = 10

In [81]:
single_step_history = single_step_model.fit(train_data_single, epochs=EPOCHS,
                                            steps_per_epoch=EVALUATION_INTERVAL,
                                            validation_data=val_data_single,
                                            validation_steps=10)

Epoch 1/10


TypeError: ignored